In [1]:
import pandas as pd
import scipy
import scipy.stats
from scipy.spatial import distance
import math
import stat
import random
from numpy.random import permutation
import numpy as np




In [2]:
#import data
PATH = "data/"
df_raw = pd.read_csv(f'{PATH}users.csv')
df = df_raw[:]


In [3]:
df.head()

,Name,Nunez-Ramirez,Scott PLC,"Miles, Padilla and Harrison","Cortez, Adams and Long","Brown, Fernandez and Stevens","Evans, Jones and Shields","Harper, Boyd and Torres",Thompson-Hall,Martin-Parker,...,Lowe PLC,Chang-Jones,"Lang, Johnson and Hebert",Blevins PLC,Marshall-Smith,"Hernandez, Espinoza and Welch",Avila Group,Baker-Smith,Martinez-Wilkins,Perez and Sons
0,Jack Murphy,0,3,1,4,0,2,3,4,2,...,1,3,1,1,1,4,2,1,0,0
1,Melissa Wagner,0,1,5,0,1,2,2,2,4,...,0,4,0,3,0,3,0,1,1,4
2,Molly Park,0,0,1,0,1,1,4,1,0,...,3,3,0,0,5,0,0,1,0,2
3,John Martinez,3,1,2,0,0,0,0,2,2,...,2,0,0,5,1,2,4,0,0,1
4,Christopher Jones,0,2,0,1,0,1,0,0,2,...,1,1,1,1,5,0,0,3,5,4


In [30]:
def stock_name_recomend(df):
"""takes as input a data frame of names with stock ratings
    askes for input for one of the names
    returns a dictionary of recommendations for how much 
    the person should buy for stocks that person doesn't hold"""
    user_liste = df['Name']
    print("Please enter the name of the Person you want to give stock recommendations. \n")
    print("The possible names are:  \n ")
    print(list(user_liste))
    Name = input()
    user = df[df['Name'] == Name]
    other_users = df.drop([user.index.values[0]])
    user_np = np.array(user)
    other_users_np = np.array(other_users)
    
    dictionary = recomend(user_np,other_users_np)
    
    stock_liste = user.columns
    
    stock_dict = {}
    
    for i in dictionary:
        stock_name = stock_liste[i]
        stock_dict[stock_name] = int(round(dictionary[i]))
        
    print("The recommendation for " + Name + " is to buy the following stocks: ")
    
    #print(stock_dict)
    
    return stock_dict
    
    
    

In [5]:
def recomend(user,other_users):
    """takes as input one arrays and a list of arrays, 
    the first element is the name all following elements are 
    The rating for a stock between 1-5, 0 stands for no rating.
    returns a dictionary with the keys being indexes of stocks 
    that the user doesn't hold and the values being a recommendation 
    for how much the user should buy of a given stock
    """
    # first for the first empty stock
    
    empty_stock_index_liste = get_empty_stock_index(user)
    
    dictionary = {}
    for i in empty_stock_index_liste: 
    
        recomendation = calculeate_recomendation_for_a_stock(user,other_users,i)
        dictionary[i] = recomendation
    
    return dictionary

In [6]:
def calculeate_recomendation_for_a_stock(user,other_users,index):
    """takes as input a user, a list of other users, and one index list of a stock that the user has not rated
    returns a recommendation between for buying between 0 and 5 of that stock"""
    
    
    five_most_correlated_users, correlation_top_five_liste = get_five_most_correlated(user,other_users,index)
    
    recomandation = formular(user,other_users,index)
    
    
    return recomandation

In [7]:
def get_five_most_correlated(user,other_users,index):
    """takes as input the other_users and the index of the stock we want recommendations on"""
    list_of_other_users_who_own_the_stock = get_list_of_other_users_who_own_the_stock(other_users,index)
    
    corr_liste = []
    for i in list_of_other_users_who_own_the_stock:
        corr = get_corr(user,i)
        corr_liste.append(corr[0])
    
    
    corr_array = np.array(corr_liste)
    index_list = (-corr_array).argsort()[:5]
    
    correlation_top_five_liste = get_top_five_correlated(corr_liste,index_list)
    
    five_most_correlated_users = get_five_most_correlated_users(other_users,index_list)
    
    
    return five_most_correlated_users, correlation_top_five_liste

In [8]:
def get_five_most_correlated_users(other_users,index_list):
    """takes as input a list of users and the indexes of these users with the highest 
    correlation to the user we want to give recommendations to
    returns a list of the five users with the highest correlation"""
    most_correlated_users = []
    for i in index_list:
        
        most_correlated_users.append(list(other_users[i]))
    
    return most_correlated_users

In [9]:
def get_top_five_correlated(corr_liste,index_liste):
    """takes as input a list of correlations and a list of the top five indexes 
    returns a list of the top five correlations"""
    correlation_top_five = []
    for i in index_liste:
        correlation_top_five.append(corr_liste[i])
    return correlation_top_five

In [10]:
def get_corr(user,i):
    """takes as input user array and i user array
    returns the pearsonr correlation"""
    i = np.array(i[1:]).astype(np.int)
    corr = scipy.stats.pearsonr(user[0][1:],i)
    return corr

In [11]:
def get_list_of_other_users_who_own_the_stock(other_users,index):
    """takes as input a list of arrays and returns a new list of 
    arrays who are not zero at the position index"""
    user_liste = []
    for i in other_users:
        if i[index] != 0:
            user_liste.append(list(i))
    user_array = np.array(user_liste)
            
    return user_array

In [12]:
def formular(user,other_users,index):
    """takes as input the user, the other users and the index of the stock we want to give a recommendation for
    returns a recommendation of how many stocks to buy"""
    five_most_correlated_users, correlation_top_five_liste = get_five_most_correlated(user,other_users,index)
    
    avr_ru = get_avr_ru(user) 
    top_calc = get_top_calc(correlation_top_five_liste,five_most_correlated_users,index)
    botom_calc = get_botom_calc(correlation_top_five_liste)
    
    recomandation = avr_ru + (top_calc)/(botom_calc)
    
    return recomandation
    

In [13]:
def get_top_calc(correlation_top_five_liste,five_most_correlated_users,index):
    """ calculates the top part of the recommendation equation"""
    top_calc = 0 
    for i in range(len(correlation_top_five_liste)):
        five_most_correlated_users[i]
        avrage_rating = get_avr_for_user(five_most_correlated_users[i])
        r_n_i = five_most_correlated_users[i][index]
        second = (r_n_i-avrage_rating)
        
        number = correlation_top_five_liste[i] * (r_n_i-avrage_rating)
        top_calc += number
    
    return top_calc

In [14]:
def get_avr_for_user(five_most_correlated_users):
    """returns the average rating over all the stocks a person owns"""
    average = np.mean(five_most_correlated_users[1:])
    
    return average 

In [15]:
def get_botom_calc(correlation_top_five_liste):
    """ calculates the bottom part of the recommendation equation"""
    botom_calc = sum(correlation_top_five_liste)
    return botom_calc

In [16]:
def get_avr_ru(user):
    """returns the average rating over all the stocks a person owns"""
    user = np.array(user[0][1:])
    avr_ru = np.mean(user)

    return avr_ru

In [17]:
def get_empty_stock_index(user):
    """takes as input a user array with name and recomandations
    returns the indexes where user recomandation == 0"""
    empty_stock_index_liste = []
    for i in range(len(user[0])):
        if user[0][i] == 0:
            empty_stock_index_liste.append(i)
    return empty_stock_index_liste
        
    

In [31]:
stock_name_recomend(df)

Please enter the name of the Person you want to give stock recomandations. 

The possible names are:  
 
['Jack Murphy', 'Melissa Wagner', 'Molly Park', 'John Martinez', 'Christopher Jones', 'Christina Munoz', 'Ana Henderson', 'Gregory Mooney', 'Francisco Blackwell', 'Nicole Peterson', 'Blake Hughes', 'Alicia Rios', 'Angela Peck', 'Earl Stevens', 'Robert Steele', 'Kathleen Lewis', 'Tammy Meyer', 'Julia Reynolds', 'April Nelson', 'Heather Woods', 'Kelsey Green', 'Christopher Cox', 'Jonathan Davis', 'Melvin Chandler', 'Samuel Moore', 'Brittany Murphy', 'Kelly Fisher', 'Hayley Irwin', 'Kevin Collins', 'Cheryl King', 'Kayla Gould', 'Michael Marquez', 'Shawn Lawrence', 'Gary Mcdonald', 'Rachel Gonzalez', 'Grant Holmes', 'Thomas Wu', 'Perry Barnett', 'Cassandra Wheeler', 'Pamela Wolf', 'Wendy Tyler', 'Victor Allen', 'Amy Sandoval', 'Lauren Preston', 'Steven Cox', 'Julie Brown', 'Stephanie Carroll', 'John Mason', 'Cristina Thomas', 'Taylor Ashley', 'Michael Brandt', 'Clifford Shannon', 'Johnn

{'Brown, Fernandez and Stevens': 2,
 'Marshall-Smith': 2,
 'Avila Group': 2,
 'Martinez-Wilkins': 2}